<a href="https://colab.research.google.com/github/yeseul106/UROP_Software-Fault-Prediction/blob/main/Ternary_Classification_JM1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib as plt

In [3]:
# 구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# 원본 csv 파일 읽어오기
JM1_original = pd.read_csv('/content/drive/MyDrive/UROP/UROP dataset/JM1_Nomalize_Severity1.csv', encoding='cp949')
print(JM1_original.shape)
JM1_original.head(5)

(10878, 22)


,LOC_BLANK,BRANCH_COUNT,LOC_CODE_AND_COMMENT,LOC_COMMENTS,CYCLOMATIC_COMPLEXITY,DESIGN_COMPLEXITY,ESSENTIAL_COMPLEXITY,LOC_EXECUTABLE,HALSTEAD_CONTENT,HALSTEAD_DIFFICULTY,HALSTEAD_EFFORT,HALSTEAD_ERROR_EST,HALSTEAD_LENGTH,HALSTEAD_LEVEL,HALSTEAD_PROG_TIME,HALSTEAD_VOLUME,NUM_OPERANDS,NUM_OPERATORS,NUM_UNIQUE_OPERANDS,NUM_UNIQUE_OPERATORS,LOC_TOTAL,3level1
0,1.000000,1.000000,0.111111,0.456395,1.000000,0.957606,0.682927,1.000000,0.369055,0.919297,1.000000,1.000000,1.000000,0.00,1.000000,1.000000,1.000000,1.000000,0.593567,0.377129,1.000000,2
1,0.000000,0.254545,0.000000,0.000000,0.270789,0.256858,0.079268,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.327812,2
2,0.366890,0.586667,0.092593,0.168605,0.569296,0.543641,0.231707,0.562323,0.356243,0.510593,0.297776,0.536178,0.571970,0.00,0.297776,0.536129,0.572658,0.585240,0.396686,0.248175,0.529788,2
3,0.082774,0.033939,0.074074,0.122093,0.038380,0.044888,0.030488,0.047096,0.189793,0.110760,0.007466,0.061967,0.081152,0.02,0.007466,0.061963,0.097650,0.071956,0.117934,0.092457,0.064226,2
4,0.024609,0.489697,0.000000,0.049419,0.859275,0.002494,0.000000,0.288244,0.177612,0.492611,0.138190,0.257885,0.240848,0.00,0.138190,0.257888,0.269116,0.225092,0.790448,1.000000,0.244987,1


In [5]:
# 입력 데이터
input_data = JM1_original.iloc[:,:21]
print("input_data shape : ",input_data.shape)

input_data shape :  (10878, 21)


In [6]:
input_data = np.asarray(input_data,dtype=np.float)

In [7]:
# 정답 레이블
y_label = JM1_original.iloc[:,21].to_numpy()
print("y_label shape : ",y_label.shape)

y_label shape :  (10878,)


# **5-fold Cross Validation**

In [8]:
from sklearn.model_selection import StratifiedKFold
from keras import models
from keras import layers
from keras.callbacks import EarlyStopping

# 5개의 파일로 쪼갬
n_fold = 5
seed = 0
skf = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

#빈 metrics 배열 // confusion matrix를 구하기
fl_score = []
accuracy = []
loss = []

In [9]:
from typing import Pattern
# 학습 자동 중단 설정
early_stopping_callback = EarlyStopping(monitor='loss', patience=100)

In [11]:
model = models.Sequential()
model.add(layers.Dense(32, input_shape = (21, )))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(32))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(32))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(3))
model.add(layers.Activation('softmax'))
model.compile(optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['categorical_accuracy'])
  
for train, test in skf.split(input_data, y_label):
  x_train, x_test = input_data[train], input_data[test]
  y_train, y_test = y_label[train], y_label[test]
  y_train = tf.keras.utils.to_categorical(y_train)
  y_test = tf.keras.utils.to_categorical(y_test)
  
  hist = model.fit(x_train, y_train, epochs=600, batch_size=32, verbose=1, callbacks=[early_stopping_callback])
  
  k_loss = "%.4f"%(model.evaluate(x_test, y_test)[0])
  k_accuracy = "%.4f"%(model.evaluate(x_test, y_test)[1])
  accuracy.append(k_accuracy)
  loss.append(k_loss)

  #예측 값 저장하기
  y_pred = model.predict(x_test).flatten()
  print(y_pred)

  from sklearn.metrics import plot_confusion_matrix
  from sklearn.metrics import f1_score
  print("y_pred.shape: ", y_pred.shape)
  print("y_label.shape: ", y_label[test].shape)

  # #평가 매트릭 출력
  # plot_confusion_matrix(model,x_test,y_test, values_format='d', cmap='Blues')
  # plt.show()
  k_f1_score = f1_score(y_label[test], y_pred,  average = 'micro' )
  f1_score.append(k_f1_score)

Epoch 1/600
272/272 [==============================] - 3s 5ms/step - loss: 0.6913 - categorical_accuracy: 0.7940
Epoch 2/600
272/272 [==============================] - 2s 6ms/step - loss: 0.5760 - categorical_accuracy: 0.8068
Epoch 3/600
272/272 [==============================] - 1s 5ms/step - loss: 0.5675 - categorical_accuracy: 0.8067
Epoch 4/600
272/272 [==============================] - 1s 5ms/step - loss: 0.5685 - categorical_accuracy: 0.8068
Epoch 5/600
272/272 [==============================] - 1s 4ms/step - loss: 0.5641 - categorical_accuracy: 0.8069
Epoch 6/600
272/272 [==============================] - 1s 4ms/step - loss: 0.5562 - categorical_accuracy: 0.8076
Epoch 7/600
272/272 [==============================] - 1s 2ms/step - loss: 0.5527 - categorical_accuracy: 0.8072
Epoch 8/600
272/272 [==============================] - 1s 2ms/step - loss: 0.5537 - categorical_accuracy: 0.8077
Epoch 9/600
272/272 [==============================] - 1s 2ms/step - loss: 0.5497 - categorical_

ValueError: ignored

In [ ]:
print("\n %.f fold accuracy:" % n_fold, accuracy)
print("모든 fold의 평균 accuracy: ", (float(accuracy[0]) + float(accuracy[1]) + float(accuracy[2]) + float(accuracy[3]) + float(accuracy[4])) / 5)

print("\n %.f fold loss:" % n_fold, loss)
print("모든 fold의 평균 loss: ", (float(loss[0]) + float(loss[1]) + float(loss[2]) + float(loss[3]) + float(loss[4])) / 5)

print("\n %.f fold f1_score:" % n_fold, f1_score)
print("모든 fold의 평균 f1_score: ", (float(f1_score[0]) + float(f1_score[1]) + float(f1_score[2]) + float(f1_score[3]) + float(f1_score[4])) / 5)
